Core Operations in OpenCV
===

In which we learn the basic operations and arithmetic operations on images.

In [ ]:
import cv2
import numpy as np

## Accessing Image Properties

The shape of an image may be accessed just as we learned with numpy arrays, `image.shape`. It returns a tuple of number of rows, columns, and channels (if the image is not grayscale). The 3 color channels are blue, green, and red in that order.

In [ ]:
image = cv2.imread('../assets/windows.jpg')

print(image.shape)

If the image is grayscale, it will only return `(ROW, COL)`. Thus, it may be used for checking if an image is grayscale or colored.

To get the image data type.

In [ ]:
print(image.dtype)

Most images will be data type `uint8`. This mean 8-bit unsigned integer which can have values from 0 to 255. A pixel value of 0 is dark and a pixel value of 255 is light.

## Access and Modify Pixel Values

We can access a pixel value by its row and column coordinates. For RGB image, it returns an array of Blue, Green, Red values. For grayscale image, just corresponding intensity is returned. Lets find the pixel value for the top left corner (0, 0) of the windows background from earlier.

In [ ]:
pixel = image[0, 0]

In [ ]:
print(pixel)

In [ ]:
# access only blue pixel
print(pixel[0])

# access only green pixel
print(pixel[1])

# access only red pixel
print(pixel[2])

As expected, this top left pixel is the sky and is therefore blue. We can see that at this pixel, blue is brighter than green is brighter than red.

## Region of Interest

In the same way we did numpy slicing before, we can slice images. Lets grab the 100 rightmost columns and display them.

In [ ]:
# Grab the last 100 columns
# You may have to resize the window
cropped = image[:, -100:]

In [ ]:
cv2.namedWindow('image', cv2.WINDOW_NORMAL)
cv2.namedWindow('cropped', cv2.WINDOW_NORMAL)
cv2.imshow('image', image)
cv2.imshow('cropped', cropped)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Splitting and Merging Image Channels

The RGB channels can be split into their individual planes when needed. It can also be merged to form RGB again.

In [ ]:
blue, green, red = cv2.split(image)

In [ ]:
cv2.imshow('blue_channel', blue)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
cv2.imshow('red_channel', red)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
cv2.imshow('green_channel', green)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
image = cv2.merge((blue, green, red))

cv2.imshow('merged_back', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

Since using `cv2.split()` and `cv2.merge()` are both computationally intensive, we can use NumPy slicing instead.

In [ ]:
blue = image[:, :, 0]

cv2.imshow('blue_channel', blue)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
green = image[:, :, 1]

cv2.imshow('green_channel', green)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
red = image[:, :, 2]

cv2.imshow('red_channel', red)
cv2.waitKey(0)
cv2.destroyAllWindows()

We can also use NumPy slicing as a way to access an image channel. When we use numpy slicing, the link still exists back to the original image. So here we can zero out the red channel by slicing out the red and setting it to zero.

In [ ]:
# modify red channel
image[:, :, 2] = 0

cv2.imshow('modified_red_channel', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

Without red, everything seems more blue-ish green.

## Color Spaces

OpenCV by default represents images in this BGR format (blue, green, red) but there are many other representations. Let's start by converting to grayscale, which is a one channel image. Each pixel value only represents total brightness. This color space is very useful if you do not care about color.

In [ ]:
# load the windows background
image = cv2.imread('../assets/windows.jpg')

image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
cv2.imshow('gray', image_gray)
cv2.waitKey(0)
cv2.destroyAllWindows()

Here we used the cvtColor functions which allows us to convert between color spaces. We can convert back to BGR, but the image will still look gray as we have lost data in the conversion.

Let's convert to a very useful color space, HSV (hue, saturation, value). Hue is the color of the pixel, ranging through the colors of the rainbow (0 - 180), saturation is how colorful the pixel is (0 grayscale - 255 neon), and value is the brightness of the pixel, very similar to grayscale. This color space is very ful when you only care about color. Let's take a peek at the saturation of the windows background and see if it makes sense.

In [ ]:
image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
hue, saturation, value = cv2.split(image_hsv)

cv2.imshow('original', image)
cv2.imshow('saturation', saturation)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Resizing

In [ ]:
# Shrink image to 100 x 100 pixels
image_small = cv2.resize(image, (100, 100))

cv2.imshow('small', image_small)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Thresholding

Thresholding performing an operation on an image based on whether the pixels values are on either side of a threshold. In all out cases, we will do a binary threshold. This means we will use it to turn our image into black an white, 0 and 255, with no in between. Here we will take anything above 200 and turn it into 255 and anything below 200 will be 0.

In [ ]:
# 1st parameter is input image
# 2nd parameter is the threshold
# 3rd parameter is the max value. 255 for uint8
# 4th parameter is the type of threshold. Binary in our case.

_, image_thresh = cv2.threshold(image_gray, 210, 255, cv2.THRESH_BINARY)

cv2.imshow('threshold', image_thresh)
cv2.waitKey(0)
cv2.destroyAllWindows()

You can see why this is useful. We just extracted each of the clouds with one operation. We will be using this a lot in the future.

## Drawing
Many times you will have a coordinate in the image but have a hard time visualizing where that is. This is where draing comes in. Lets draw a circle around the coordinate 145, 278.

In [ ]:
center = (145, 278)
radius = 10
color = (0, 0, 255)
thickness = 2

image_annotated = image.copy()
cv2.circle(image_annotated, center, radius, color, thickness)
cv2.imshow('annotated', image_annotated)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Processing Video
Processing video is as simple as applying operations on each frame as you get it. Let's make the pong video from earlier blue, and then draw a green circle in the center.

In [ ]:
import cv2
import numpy as np

cap = cv2.VideoCapture("../assets/pong.mp4")

while True:
    ret, frame = cap.read()

    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break

    # Frame is the current image from the video.
    # To make the image blue, we can eliminate all the reg and green in the image.
    frame[:, :, 1] = 0  # Eliminate green 
    frame[:, :, 2] = 0  # Eliminate red

    # To draw a circle in the center, we first need to know how many pixels tall and wide the image is. 
    height, width, channels = frame.shape

    # // is integer division. 5 // 2 = 2
    center = (width // 2, height // 2)
    radius = 10
    color = (0, 255, 0)
    thickness = 2

    cv2.circle(frame, center, radius, color, thickness)
    
    cv2.imshow('Pong.mp4', frame)
    
    # If a key was pressed
    if cv2.waitKey(20) > 0:
        break
        
cap.release()
cv2.destroyAllWindows()